In [ ]:
import tushare as ts
from dotenv import load_dotenv
import os
Tushare_API_Key=os.getenv("Tushare_API_Key")

In [ ]:
import sqlite3
import pandas as pd
import tushare as ts
import time
from dotenv import load_dotenv
import os
# ==============================
# 1. Config
# ==============================
TUSHARE_TOKEN = os.getenv("Tushare_API_Key")
ts.set_token(TUSHARE_TOKEN)
pro = ts.pro_api()

DB_PATH = "stocks.db"
SYMBOLS=pd.read_csv("target_instruments/csi300.csv")['con_code'].unique().tolist()

START_DATE = "20140101"
END_DATE = "20250930"

# ==============================
# 2. SQLite Init
# ==============================
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS stock_data (
    _id TEXT PRIMARY KEY,           -- ts_code + trade_date
    symbol TEXT,
    date TEXT,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume REAL,
    adj_factor REAL
)
""")
conn.commit()

# ==============================
# 3. Fetch & Store
# ==============================
for symbol in SYMBOLS:
    print(f"Fetching: {symbol}")
    time.sleep(1)  # to respect API rate limits
    # --- daily price ---
    daily_df = pro.daily(
        ts_code=symbol,
        start_date=START_DATE,
        end_date=END_DATE,
        fields="ts_code,trade_date,open,high,low,close,vol"
    )

    # --- adj factor ---
    adj_df = pro.adj_factor(
        ts_code=symbol,
        start_date=START_DATE,
        end_date=END_DATE,
        fields="ts_code,trade_date,adj_factor"
    )

    # --- merge on ts_code + trade_date ---
    merged = pd.merge(
        daily_df,
        adj_df,
        on=["ts_code", "trade_date"],
        how="left"
    ).drop_duplicates(subset=["ts_code", "trade_date"])

    # --- rename columns to match table ---
    merged.rename(columns={
        "ts_code": "symbol",
        "trade_date": "date",
        "vol": "volume"
    }, inplace=True)

    # --- generate _id for uniqueness ---
    merged["_id"] = merged["symbol"] + "_" + merged["date"]

    # --- insert into SQLite ---
    merged.to_sql(
        "stock_data",
        conn,
        if_exists="append",
        index=False,
        method="multi"
    )

conn.commit()
conn.close()
print("✅ Data fetch and store complete.")
